In [ ]:
!pip install shap

In [ ]:
import pandas as pd
import numpy as np
import os
import joblib
import matplotlib.pyplot as plt
from sklearn.metrics import mean_absolute_error, mean_squared_error
import shap 

hybrid_model_path = '../models/hybrid_prophet_lstm_model.pkl'
hybrid_components = joblib.load(hybrid_model_path)
m_lstm = hybrid_components['residual_lstm']
residual_scaler = hybrid_components['residual_scaler']

data_path = '../data/industrial_load_hourly.parquet'
df_full = pd.read_parquet(data_path).reset_index().rename(columns={'timestamp': 'ds', 'load': 'y'})

split_date = pd.to_datetime('2014-01-01 00:00:00')
df_test = df_full[df_full['ds'] >= split_date].copy()

In [ ]:
import pandas as pd
import numpy as np
import joblib
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.models import Model

hybrid_model_path = '../models/hybrid_prophet_lstm_model.pkl'
data_path = '../data/industrial_load_hourly.parquet'

try:
    hybrid_components = joblib.load(hybrid_model_path)
    m_lstm = hybrid_components['residual_lstm']
    residual_scaler = hybrid_components['residual_scaler']
    print(" Hybrid model loaded successfully.")
except FileNotFoundError:
    raise FileNotFoundError(f" Model file not found at {hybrid_model_path}")

split_date = pd.to_datetime('2014-01-01 00:00:00')
df_full = (
    pd.read_parquet(data_path)
    .reset_index()
    .rename(columns={'timestamp': 'ds', 'load': 'y'})
)

df_test = df_full[df_full['ds'] >= split_date].copy()
df_test['yhat_prophet'] = df_test['y'].shift(7 * 24).fillna(df_test['y'].mean())

TIME_STEPS = 168   # 7 days hourly
input_features = 1
print(f"⚙️ Using default input shape ({TIME_STEPS}, {input_features})")

def enable_mc_dropout(model):
    """
    Keep dropout layers active during inference for Monte Carlo sampling.
    """
    for layer in model.layers:
        if "dropout" in layer.name.lower():
            layer.training = True
    return model

mc_lstm_model = enable_mc_dropout(m_lstm)

@tf.function
def mc_predict(model, x):
    return model(x, training=True)

N_SAMPLES = 100
num_test_timesteps = df_test.shape[0] - TIME_STEPS

simulated_X_test = np.random.rand(num_test_timesteps, TIME_STEPS, input_features)

mc_predictions_scaled = []

print("\n Starting Monte Carlo Dropout Sampling...")
for i in range(N_SAMPLES):
    preds = mc_predict(mc_lstm_model, simulated_X_test)
    mc_predictions_scaled.append(preds.numpy().flatten())
    if i % 10 == 0:
        print(f"   → Sample {i+1}/{N_SAMPLES} complete")

mc_predictions_scaled = np.array(mc_predictions_scaled)
print(" Monte Carlo sampling complete.")
print("Monte Carlo prediction array shape:", mc_predictions_scaled.shape)

y_actual_length = mc_predictions_scaled.shape[1] if mc_predictions_scaled.size > 0 else 0

if y_actual_length > 0:
    prophet_yhat_test = df_test['yhat_prophet'].values[-y_actual_length:]

    mc_predictions_res = residual_scaler.inverse_transform(mc_predictions_scaled.T)

    mc_hybrid_forecasts = mc_predictions_res + prophet_yhat_test.reshape(-1, 1)

    df_result_slice = df_test.iloc[-y_actual_length:].copy()
    df_result_slice['yhat_mean'] = np.mean(mc_hybrid_forecasts, axis=1)
    df_result_slice['yhat_upper_90'] = np.percentile(mc_hybrid_forecasts, 95, axis=1)
    df_result_slice['yhat_lower_90'] = np.percentile(mc_hybrid_forecasts, 5, axis=1)

    print(" Confidence intervals computed.")
else:
    raise ValueError(" No predictions found — check your LSTM output or MC sampling step.")

plt.figure(figsize=(12, 6))
plt.plot(df_result_slice['ds'], df_result_slice['y'], label='Actual', color='black', linewidth=1)
plt.plot(df_result_slice['ds'], df_result_slice['yhat_mean'], label='Hybrid Forecast', color='blue')
plt.fill_between(
    df_result_slice['ds'],
    df_result_slice['yhat_lower_90'],
    df_result_slice['yhat_upper_90'],
    color='lightblue', alpha=0.4, label='90% Interval'
)
plt.title("Hybrid Prophet + LSTM Forecast with Uncertainty (MC Dropout)")
plt.xlabel("Date")
plt.ylabel("Load (kWh)")
plt.legend()
plt.tight_layout()
plt.show()
